<div class="alert alert-block alert-info">
<b>
Note:
</b> 
Following along with the book but condensing and making my own changes.
</div>


In [5]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [6]:
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

Tensorflow version: 2.5.0
Keras version: 2.5.0


# Generating Shakespearean text using a Character RNN

### Creating the training dataset

In [24]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [25]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts([shakespeare_text])

In [26]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [27]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [ ]:
# Number of distinct characters
max_id = len(tokenizer.word_index)

# Set values to 0 index
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

<div class="alert alert-block alert-info">
<b>
Note:
</b> 
For some reason the attribute `tokenizer.document_count` evaluates to 1 now, so we have to get the total number of words a slightly different way.
</div>


In [34]:
# Total character count
dataset_size = len(encoded)

In [45]:
# Create training dataset in TensorFlow
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### Chopping the sequential dataset into multiple windows

In [46]:
# Create windows
window_length = 101
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)

dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map(lambda x_batch, y_batch: (tf.one_hot(x_batch, depth = max_id), y_batch))
dataset = dataset.prefetch(1)

### Building and training the Char-RNN model

In [53]:
model = keras.models.Sequential([keras.layers.GRU(128, return_sequences = True, input_shape = [None, max_id], 
#                                                  dropout = 0.2, recurrent_dropout = 0.2),
                                                  dropout = 0.2),
                                keras.layers.GRU(128, return_sequences = True, input_shape = [None, max_id], 
#                                                  dropout = 0.2, recurrent_dropout = 0.2), 
                                                 dropout = 0.2),
                                keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation = 'softmax'))])
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')
history = model.fit(dataset, epochs = 1)

2021-10-12 12:04:36.525013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-12 12:04:37.473087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-12 12:04:37.967199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-12 12:04:38.484194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-12 12:04:43.945049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


31368/31368 [==============================] - 6338s 202ms/step - loss: 2.86106314s 201ms/step - loss: 2.86 - 631


### Using the Char-RNN model

In [54]:
6338/7150

0.8864335664335664

In [55]:
def preprocess(texts):
    x = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(x, max_id)

In [57]:
x_new = preprocess(['How are yo'])
# y_pred = model.predict_classes(x_new)
y_pred = np.argmax(model.predict(x_new), axis=-1)
tokenizer.sequences_to_texts(y_pred + 1)[0][-1]

'r'

### Generating fake Shakespearean text

In [1]:
def next_char(text, temperature = 1):
    x_new = preprocess([text])
    y_proba = model.predict(x_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [2]:
def complete_text(text, n_chars = 50, temperature = 1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
print(complete_text('t', temperature = 0.2))

In [ ]:
print(complete_text('w', temperature = 1))

In [ ]:
print(complete_text('w', temperature = 2))

# Stateful RNN

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift = 100, drop_remainder = True)
dataset = dataset.flat_map(lambda window: window.batch(100))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda x_batch, y_batch: (tf.one_hot(x_batch, depth = max_id), y_batch))
dataset = dataset.prefetch(1)

In [ ]:
model = keeras.models.Sequential([keras.layers.GRU(128, return_sequences = True, 
                                                   stateful = True, dropout = 0.2, 
                                                   batch_input_shape = [batch_size, None, max_id]), 
                                  keras.layers.GRU(128, return_sequences = True, stateful = True, 
                                                   dropout = 0.2), 
                                  keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation = 'softmax'))])

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')
model.fit(dataset, epochs = 50, callback = [ResetStatesCallback()])

# Sentiment analysis